### 一、写在前面

深度神经网络已经在各种视觉识别问题上取得了最先进的性能。尽管取得了极大成功，深度神经网络很容易遭受输入上微小和不可察觉的干扰导致的误分类（这些输入也被称作对抗样本），深度模型的安全问题也在业内引起了不少担忧。为了发现目标检测模型的脆弱性，我们将通过采用COCO数据集，其中包含20类物体。任务是通过向原始图像中添加对抗补丁/Adversarial Patch的方式，使得典型的目标检测模型不能够检测到图像中的物体，绕过目标定位。希望实现在图片上添加的补丁数量、修改的像素和模型识别到的包围盒越少，则代表攻击更加成功，得分则越高。

简单来说，所谓对抗补丁，即向原始样本中添加一些人眼无法察觉的噪声，这样的噪声不会影响人类的识别，但是却很容易欺骗模型，使其作出与正确答案完全不同的判定。如下图所示，该攻击方式能欺骗目标检测模型：

<div align=center>
    <img src="https://ai-studio-static-online.cdn.bcebos.com/539713eae8414103853ff8309543b33593507c89a91a436da961c1f8be7156c3" width="500">
</div>

将肉眼不可见的细小扰动加入到原图中，网络识别出粗，甚至有99.3% 的概率认为这是长臂猿。

这样的对抗攻击，从模型角度分为：

* 白盒攻击：攻击者能够获知机器学习所使用的算法，以及算法所使用的参数。

* 黑盒攻击：攻击者并不知道机器学习所使用的算法和参数，但攻击者仍能与机器学习的系统有所交互，比如可以通过传入任意输入观察输出，判断输出。

以下我们尝试黑盒攻击。


### 二、相关准备

#### 2.1 MSCOCO

加载[MSCOCO 2017 Test > ObjectDetection](https://aistudio.baidu.com/datasetdetail/298599)。该数据集从MSCOCO 2017测试数据集中有条件的筛选了1000张图像，这些图像不会包含过多或者过少的检测框（至少会有一个检测框），并且检测框的面积相对于整图不会太小。每张图都被resize到500 * 500的大小，并以.png的格式存储。

<div align=center>
    <img src="https://ai-studio-static-online.cdn.bcebos.com/c809355b871a43cda67c485109e310efeeeb511f5e014504b28d6276762a80ad" width="200" height="200">
    <img src="https://ai-studio-static-online.cdn.bcebos.com/72ed25ab18164401a5581eb73b147a1de43c5d026f6f4696adf65219033ca370" width="200" height="200">
    <img src="https://ai-studio-static-online.cdn.bcebos.com/d64f159ec13b417c8a462ccbe0b13548798353c3a9714addb1602d9045613361" width="200" height="200">
    <img src="https://ai-studio-static-online.cdn.bcebos.com/ce3c9d5f12124e15b48a1f0a2dc4871782408777c5744f109898f9634711a060" width="200" height="200">
</div>


In [1]:

%cd data/data298599
!ls -l && rm -rf images && unzip -q images.zip && clear
!ls -l images/*.png|wc -l
%cd /home/aistudio


/home/aistudio/data/data298599
total 384324
drwxr-xr-x 2 aistudio aistudio     24576 Jun 15  2020 images
-rwxrwxrwx 1 aistudio aistudio 393523182 Oct 29 03:06 images.zip
1000
/home/aistudio


#### 2.2 环境依赖

* 安装PaddlePaddle
```shell
# cpu
python -m pip install paddlepaddle==3.0.0b1 -i https://www.paddlepaddle.org.cn/packages/stable/cpu/

# gpu，该命令仅适用于 CUDA 版本为 11.8 的机器环境
python -m pip install paddlepaddle-gpu==3.0.0b1 -i https://www.paddlepaddle.org.cn/packages/stable/cu118/

# gpu，该命令仅适用于 CUDA 版本为 12.3 的机器环境
python -m pip install paddlepaddle-gpu==3.0.0b1 -i https://www.paddlepaddle.org.cn/packages/stable/cu123/
```

* 安装PaddleX
```shell
pip install https://paddle-model-ecology.bj.bcebos.com/paddlex/whl/paddlex-3.0.0b1-py3-none-any.whl --user
```

* 其他
```shell
pip install cmaes
```

可执行build.sh


In [2]:
import paddle
import paddlex
print(paddle.__version__, paddlex.__version__)


3.0.0-beta1 3.0.0.beta1


### 三、通用目标检测

我们通过PaddleX加载*object_detection*产线作为通用目标检测模型，在随后的对抗攻击中，将其视为黑盒模型。


In [3]:

from paddlex import create_pipeline
pipeline = create_pipeline(pipeline="object_detection")

output = pipeline.predict([
    "work/demo/1013.png", 
    "work/demo/1016.png",
    "work/demo/1027.png", 
    "work/demo/1043.png",
])
for res in output:
    res.print() ## 打印预测的结构化输出
    res.save_to_img("work/output/") ## 保存结果可视化图像
    res.save_to_json("work/output/") ## 保存预测的结构化输出


Using official model (PicoDet-S), the model files will be be automatically downloaded and saved in /home/aistudio/.paddlex/official_models.
{'input_path': 'work/demo/1013.png', 'boxes': [{'cls_id': 1, 'label': 'bicycle', 'score': 0.740688145160675, 'coordinate': [124.43766, 319.54004, 261.32574, 453.48654]}, {'cls_id': 0, 'label': 'person', 'score': 0.6562690138816833, 'coordinate': [170.30833, 240.64005, 280.00446, 400.9371]}, {'cls_id': 0, 'label': 'person', 'score': 0.6149351596832275, 'coordinate': [74.23787, 252.88297, 222.41718, 446.02075]}, {'cls_id': 0, 'label': 'person', 'score': 0.570392906665802, 'coordinate': [122.666084, 252.68002, 244.31424, 448.47565]}]}
The result has been saved in work/output/1013.png.
The result has been saved in work/output/1013.json.
{'input_path': 'work/demo/1016.png', 'boxes': [{'cls_id': 0, 'label': 'person', 'score': 0.9601375460624695, 'coordinate': [85.41749, 18.468357, 452.5077, 499.85214]}, {'cls_id': 63, 'label': 'laptop', 'score': 0.771952

### 四、对抗样本

以下，我们随机生成一个例子：

In [6]:

def Example():
    import cv2 as cv
    import numpy as np

    # 图片样例
    demo = cv.imread("work/demo/demo.png")
    print(f"demo {demo.mean():.4f}")

    # 随机扰动
    rand = np.random.randint(0, 255, size=(500, 500, 3))
    print(f"rand {np.abs(rand).mean():.4f}")
    cv.imwrite("work/demo/demo-rand.png", rand)

    # 添加扰动
    demr = demo + rand
    print(f"demr {demr.mean():.4f}")
    cv.imwrite("work/demo/demr.png", demr)

    # 黑盒推理
    from paddlex import create_pipeline
    pipeline = create_pipeline(pipeline="object_detection")

    output = pipeline.predict([
        "work/demo/demo.png",
        "work/demo/demr.png",
    ])
    for res in output:
        res.print() ## 打印预测的结构化输出
        res.save_to_img("work/output/") ## 保存结果可视化图像
        res.save_to_json("work/output/") ## 保存预测的结构化输出

Example()


Using official model (PicoDet-S), the model files will be be automatically downloaded and saved in /home/aistudio/.paddlex/official_models.


demo 111.2813
rand 127.0713
demr 238.3527


{'input_path': 'work/demo/demo.png', 'boxes': [{'cls_id': 0, 'label': 'person', 'score': 0.7875499725341797, 'coordinate': [65.44652, 253.39186, 217.66763, 451.90607]}, {'cls_id': 1, 'label': 'bicycle', 'score': 0.7632611393928528, 'coordinate': [123.24549, 319.33823, 265.9292, 456.16003]}, {'cls_id': 0, 'label': 'person', 'score': 0.7594175338745117, 'coordinate': [179.5725, 233.58403, 277.63705, 401.70193]}]}
The result has been saved in work/output/demo.png.
The result has been saved in work/output/demo.json.
{'input_path': 'work/demo/demr.png', 'boxes': [{'cls_id': 74, 'label': 'clock', 'score': 0.5721063017845154, 'coordinate': [293.21692, 19.47118, 472.84375, 153.48776]}]}
The result has been saved in work/output/demr.png.
The result has been saved in work/output/demr.json.


<div align=center>
    <img src="https://ai-studio-static-online.cdn.bcebos.com/c7a142c0ba7b4c66816779a21610bca81dea365d561746bcb47f236410b92db8" width="250" height="250">
    +
    <img src="https://ai-studio-static-online.cdn.bcebos.com/c05361a671074ce9a4eeb2479c7ab7664feff1f14c3e4a17bf0d2eec2eafdc95" width="250" height="250">
    =
    <img src="https://ai-studio-static-online.cdn.bcebos.com/7d87b705baf34189b1aafd47d91ca1f2b8f998db027e4970968530c190eeca21" width="250" height="250">
</div>

可以看到加了这么大的扰动(np.abs(x).mean() > 100)之后，尽管图片有点面目全非，但人眼还是能依稀识别到。

而对于此目标检测模型已基本识别不到人和自行车了，甚至会误把右上角的交通标识识别为时钟。


### 五、黑盒优化

我们用一种简单快捷的方法来完成这次攻击：

* Covariance matrix adaptation evolution strategy: CMA-ES

一种进化算法，也就是一种黑箱优化算法。

我们尝试将需要生成的补丁压缩到N * N * 3这样的一维矩阵上，通过CMA算法计算能使得损失函数最小化的补丁。

这里我们定义损失函数为l1 * 0.1 + l2 * 0.9

* 其中，l1为补丁大小，取其绝对值的平均值，l2为打补丁后新图片平均识别score

希望能实现最小化补丁但我们要攻击的目标检测模型几乎检测不到内容。


In [7]:

import os
import cv2 as cv
import numpy as np
# from tqdm import tqdm
from cmaes import CMA
from paddlex import create_pipeline
pipeline = create_pipeline(pipeline="object_detection")

base = cv.imread("work/demo/demo.png")
# print(base.shape)

# 参数定义
# 图像大小
N = 5
# 图像尺寸
S = N * N * 3
# 迭代次数和种群大小
R, P = 300, 10

def floss(x, save=False):
    _x = np.resize(x.reshape((N, N, 3)), (500, 500, 3))
    cv.imwrite("work/demo/test_x.png", _x)
    cv.imwrite("work/demo/test_r.png", base + _x)
    
    n, score = 0, 0
    output = pipeline.predict("work/demo/test_r.png")
    for res in output:
        if save:
            res.save_to_img("work/output/")
        for box_res in res["boxes"]:
            n += 1
            score += box_res["score"]
    return np.abs(_x).mean(), score/n if n > 0 else 0

# 定义CMA优化器
OPT = CMA(
    mean=np.zeros(S),
    sigma=10, 
    bounds=np.array([[-255, 255] for _ in range(S)]), 
    population_size=P
)

loss_min = np.inf
for rI in range(R):
    solutions = []
    for rJ in range(P):
        x = OPT.ask()
        l1, l2 = floss(x)
        # 定义loss函数
        loss = l1 * 0.1 + l2 * 0.9
        solutions.append((x, loss))

        _r = "\r"
        if loss_min > loss:
            loss_min = loss
            # floss(x, save=True)
            # os.system("cp work/demo/test_r.png work/demo/test_min.png")
            _r = "\tMIN\n"

        print(f"\t#{rI:5d}/{rJ:5d}\tloss:{loss:.6f}\tl1:{l1:.6f}\tl2:{l2:.6f}{_r}", end="")
    OPT.tell(solutions)


Using official model (PicoDet-S), the model files will be be automatically downloaded and saved in /home/aistudio/.paddlex/official_models.


	#    0/    0	loss:1.543785	l1:8.405434	l2:0.781380	MIN
	#    0/    1	loss:1.535243	l1:8.509549	l2:0.760320	MIN
	#    0/    2	loss:1.521629	l1:8.130141	l2:0.787349	MIN
	#    0/    6	loss:1.414790	l1:7.106123	l2:0.782420	MIN
	#   12/    7	loss:1.204070	l1:12.040696	l2:0.000000	MIN
	#   42/    1	loss:1.199241	l1:11.992408	l2:0.000000	MIN
	#   43/    7	loss:1.154182	l1:11.541816	l2:0.000000	MIN
	#   71/    5	loss:1.146530	l1:11.465298	l2:0.000000	MIN
	#   74/    8	loss:1.133101	l1:11.331010	l2:0.000000	MIN
	#   76/    2	loss:1.080444	l1:10.804442	l2:0.000000	MIN
	#   84/    0	loss:1.063679	l1:10.636786	l2:0.000000	MIN
	#   89/    8	loss:1.062626	l1:10.626258	l2:0.000000	MIN
	#   94/    0	loss:1.033305	l1:10.333052	l2:0.000000	MIN
	#   96/    9	loss:0.978162	l1:9.781625	l2:0.000000	MIN
	#   97/    8	loss:0.977157	l1:9.771575	l2:0.000000	MIN
	#   98/    6	loss:0.974220	l1:9.742201	l2:0.000000	MIN
	#   98/    7	loss:0.971463	l1:9.714630	l2:0.000000	MIN
	#   99/    4	loss:0.957249	l1:9.572489

In [8]:
# 黑盒推理
from paddlex import create_pipeline
pipeline = create_pipeline(pipeline="object_detection")

output = pipeline.predict(
    "work/demo/test_r.png"
)
for res in output:
    res.print() ## 打印预测的结构化输出
    res.save_to_img("work/output/") ## 保存结果可视化图像
    res.save_to_json("work/output/") ## 保存预测的结构化输出


Using official model (PicoDet-S), the model files will be be automatically downloaded and saved in /home/aistudio/.paddlex/official_models.
{'input_path': 'work/demo/test_r.png', 'boxes': []}
The result has been saved in work/output/test_r.png.
The result has been saved in work/output/test_r.json.


<div align=center>
    <img src="https://ai-studio-static-online.cdn.bcebos.com/c7a142c0ba7b4c66816779a21610bca81dea365d561746bcb47f236410b92db8" width="250" height="250">
    +
    <img src="https://ai-studio-static-online.cdn.bcebos.com/6b76cc095b3847fa89381f82a9f24f70af71c17d6ffa44f5a8228b9bdc724d88" width="250" height="250">
    =
    <img src="https://ai-studio-static-online.cdn.bcebos.com/1166b47e466d400c8854e003fee9d90eaf5e5f5341ae421ea160432e60301caf" width="250" height="250">
</div>

由此我们得到较小的扰动(np.abs(x).mean() < 5)之后，人眼甚至会觉得几乎是同一张照片，只是多了一些水纹。

而对于此目标检测模型就几乎都识别不出来任何东西了。

### 六、最后

当然这还远不远不够，不过这是一个很好的补丁体验。

这样的模拟黑盒攻击我们还会继续，

下一版，我们会用更加复杂的算法去得到更加精细的结果！
